In [27]:
import tensorflow as tf
from tensorflow import keras
import kerastuner as kt

Utilizando keras tuner para modelo de classificação
https://www.tensorflow.org/tutorials/keras/keras_tuner

In [3]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()
# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

In [5]:
### Definindo modelo
#def model_builder(hp):
#    model = keras.Sequential()
#    model.add(keras.layers.Flatten(input_shape=(28, 28)))
#
#    # Selecionando número de unidades na primeira camada, entre 32 e 512, ao passo de 32
#    hp_units = hp.Int('units', min_value = 32, max_value = 512, step = 32)
#    model.add(keras.layers.Dense(units = hp_units, activation = 'relu'))
#    model.add(keras.layers.Dense(10))
#
#    # Selecionando valor do learning_rate entre 0,01 , 0,001 , 0,0001
#    hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4]) 
#
#    model.compile(optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate),
#                loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True), 
#                metrics = ['accuracy'])
#
#    return model

###### Sintonizadores disponíveis
RandomSearch -> 

Hyperband ->

BayesianOptimization ->

Sklearn ->

In [28]:
#tuner_hpb = kt.Hyperband(model_builder,
#                     objective = 'val_accuracy', 
#                     max_epochs = 10,
#                     factor = 3,
#                     directory = 'my_dir',
#                     project_name = 'intro_to_kt')
#
#tuner_bay = kt.BayesianOptimization(model_builder, 
#                                    objective = 'val_accuracy', 
#                                    max_trials = 10)

In [29]:
#tuner_hpb.search(img_train, label_train, epochs = 10, validation_data = (img_test, label_test))

## Get the optimal hyperparameters
#best_hps = tuner_hpb.get_best_hyperparameters(num_trials = 1)[0]

#print(f"""
#The hyperparameter search is complete. The optimal number of units in the first densely-connected
#layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
#is {best_hps.get('learning_rate')}.
#""")

In [26]:
#tuner_bay.search(img_train, label_train, epochs = 10, validation_data = (img_test, label_test))

# Get the optimal hyperparameters
#best_hps = tuner_bay.get_best_hyperparameters(num_trials = 1)[0]

#print(f"""
#The hyperparameter search is complete. The optimal number of units in the first densely-connected
#layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
#is {best_hps.get('learning_rate')}.
#""")

In [30]:
# importando dependências
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time 
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor

In [43]:
#Recolhendo dados do facebook e preparando dataset

import pandas_datareader.data as web
import datetime as dt

end = dt.datetime(2020, 6, 1)
start = dt.datetime(2019, 1, 1)

df = web.DataReader("FB", 'yahoo', start, end)

df = df.reset_index()
df = df.drop(columns=['Open','Date','High','Low','Volume','Adj Close'])
df = df.rename(columns={'Close': 'Close 0'})

def window (df, w):
    for i in range(1,w):
        df['Close '+str(i)] = df['Close '+str(i-1)].shift(1)
    return df
        
df = window(df,5)
df = df.rename(columns={'Close 0': 'Target'})
df.dropna(inplace=True)

#Separando dados de treino e teste
X = df.loc[:, ['Close 1','Close 2','Close 3','Close 4']]
y = df.loc[:, 'Target']

X = X.astype(float)
y = y.astype(float)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

In [61]:
def model_builder__(hp):
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(X_train.shape[1],)))

    # Selecionando número de unidades na primeira camada, entre 32 e 512, ao passo de 32
    hp_units = hp.Int('units', min_value = 32, max_value = 512, step = 32)
    model.add(keras.layers.Dense(units = hp_units, activation = 'relu'))
    model.add(keras.layers.Dense(1, activation = 'relu'))

    # Selecionando valor do learning_rate entre 0,01 , 0,001 , 0,0001
    hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4]) 

    model.compile(optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate),
                loss = 'mse')

    return model

In [85]:
__tuner_hpb = kt.Hyperband(model_builder,
                     objective = 'loss', 
                     max_epochs = 10,
                     factor = 3,
                          overwrite=True)

__tuner_bay = kt.BayesianOptimization(model_builder, 
                                    objective = 'loss', 
                                    max_trials = 10)

tuner_rd = kt.RandomSearch(model_builder, 
                           objective = 'loss', 
                           max_trials = 10)


In [77]:
__tuner_hpb.search(X_train, y_train, epochs = 10, validation_data = (X_test, y_test))

# Get the optimal hyperparameters
best_hps = __tuner_hpb.get_best_hyperparameters(num_trials = 1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

INFO:tensorflow:Oracle triggered exit

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 288 and the optimal learning rate for the optimizer
is 0.01.



In [81]:
__tuner_bay.search(X_train, y_train, epochs = 10, validation_data = (X_test, y_test))

# Get the optimal hyperparameters
best_hps = __tuner_bay.get_best_hyperparameters(num_trials = 1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

Trial 10 Complete [00h 00m 02s]
loss: 35211.0859375

Best loss So Far: 30.458227157592773
Total elapsed time: 00h 00m 40s
INFO:tensorflow:Oracle triggered exit

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 512 and the optimal learning rate for the optimizer
is 0.01.



In [82]:
tuner_rd.search(X_train, y_train, epochs = 10, validation_data = (X_test, y_test))

# Get the optimal hyperparameters
best_hps = tuner_rd.get_best_hyperparameters(num_trials = 1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

Trial 10 Complete [00h 00m 04s]
loss: 30.593875885009766

Best loss So Far: 27.611289978027344
Total elapsed time: 00h 00m 41s
INFO:tensorflow:Oracle triggered exit

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 64 and the optimal learning rate for the optimizer
is 0.01.

